## Adding Human Interaction  

In the previous notebook, we created a workflow consisting of two agents, both powered by an LLM—one acting as the writer and the other reviewing the content, providing feedback, or granting approval.  

However, what if we want a human to evaluate the poem instead of relying on an LLM? In this case, we can use a special agent designed to function as a human proxy, allowing direct human intervention in the workflow.  


In [2]:
import sys
sys.path.append("..")
from model_clients.azure import get_model
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from rich import print

In the following cell, we'll define two agents, one, as before its in charge of creating the content,
 while the other is a special `UserProxyAgent` 
 that acts as any other agent but will handle the input through a dedicate input function.  
 
In this specific example it will map to Python's `input`function to allow manual entry.

In [3]:
# Create the writer agent.
writer_agent = AssistantAgent(
    "writer",
    model_client=get_model(),
    system_message="You are a helpful AI assistant.",
)

# Create the human/user agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.


Now we create the team and run it.  

After you run the next cell, type your reply or 'APPROVE' in the prompt that appears at the top 😉.

In [5]:
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([writer_agent, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
result= await Console(stream)
print("[cyan][bold]--- Conversation complete! ---[/bold][/cyan]")
print(result)

# Note how a team (as an agent)can save its state to a JSON object.
# This JSON object can be used to re-create the team's state later using the `load_state` method.
json=(await team.save_state())
print(json)

---------- user ----------
Write a 4-line poem about the ocean.
---------- writer ----------
Blue depths cradle secrets untold,  
Where moonlit waves gently unfold,  
The ocean's voice, a timeless song,  
In its embrace, we find we belong.
---------- user_proxy ----------
make it more funny
---------- writer ----------
The ocean's waves like disco beats sway,  
Fish throwing parties in crowded bay,  
Crabs tap dance in sandy nightclub halls,  
As seagulls gossip in surfboard stalls.
---------- user_proxy ----------
APPROVE


--- Conversation complete! ---

TaskResult(
    messages=[
        TextMessage(
            source='user',
            models_usage=None,
            metadata={},
            content='Write a 4-line poem about the ocean.',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=88, completion_tokens=36),
            metadata={},
            content="Blue depths cradle secrets untold,  \nWhere moonlit waves gently unfold,  \nThe ocean's voice,
a timeless song,  \nIn its embrace, we find we belong.",
            type='TextMessage'
        ),
        UserInputRequestedEvent(
            source='user_proxy',
            models_usage=None,
            metadata={},
            request_id='c64fed00-779c-40d8-a59a-f5e9abe72dfa',
            content='',
            type='UserInputRequestedEvent'
        ),
        TextMessage(
            source='user_proxy',
            models_usage=None,
            metadata={},
            content='make it more funny',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=139, completion_tokens=39),
            metadata={},
            content="The ocean's waves like disco beats sway,  \nFish throwing parties in crowded bay,  \nCrabs tap
dance in sandy nightclub halls,  \nAs seagulls gossip in surfboard stalls.",
            type='TextMessage'
        ),
        UserInputRequestedEvent(
            source='user_proxy',
            models_usage=None,
            metadata={},
            request_id='364b2dff-aa47-4862-9236-37a996dfb3bc',
            content='',
            type='UserInputRequestedEvent'
        ),
        TextMessage(source='user_proxy', models_usage=None, metadata={}, content='APPROVE', type='TextMessage')
    ],
    stop_reason="Text 'APPROVE' mentioned"
)

{
    'type': 'TeamState',
    'version': '1.0.0',
    'agent_states': {
        'writer': {
            'type': 'ChatAgentContainerState',
            'version': '1.0.0',
            'agent_state': {
                'type': 'AssistantAgentState',
                'version': '1.0.0',
                'llm_context': {
                    'messages': [
                        {
                            'content': 'Write a 4-line poem about the ocean.',
                            'source': 'user',
                            'type': 'UserMessage'
                        },
                        {
                            'content': "Endless waves whisper ancient tales,  \nBeneath the sun's golden embrace,  
\nA symphony of life in azure sails,  \nWhere dreams dance in the sea's vast space.",
                            'thought': None,
                            'source': 'writer',
                            'type': 'AssistantMessage'
                        },
                        {
                            'content': 'Write a 4-line poem about the ocean.',
                            'source': 'user',
                            'type': 'UserMessage'
                        },
                        {
                            'content': "Blue depths cradle secrets untold,  \nWhere moonlit waves gently unfold,  
\nThe ocean's voice, a timeless song,  \nIn its embrace, we find we belong.",
                            'thought': None,
                            'source': 'writer',
                            'type': 'AssistantMessage'
                        },
                        {'content': 'make it more funny', 'source': 'user_proxy', 'type': 'UserMessage'},
                        {
                            'content': "The ocean's waves like disco beats sway,  \nFish throwing parties in 
crowded bay,  \nCrabs tap dance in sandy nightclub halls,  \nAs seagulls gossip in surfboard stalls.",
                            'thought': None,
                            'source': 'writer',
                            'type': 'AssistantMessage'
                        }
                    ]
                }
            },
            'message_buffer': [
                {
                    'source': 'user_proxy',
                    'models_usage': None,
                    'metadata': {},
                    'content': 'APPROVE',
                    'type': 'TextMessage'
                }
            ]
        },
        'user_proxy': {
            'type': 'ChatAgentContainerState',
            'version': '1.0.0',
            'agent_state': {'type': 'BaseState', 'version': '1.0.0'},
            'message_buffer': []
        },
        'RoundRobinGroupChatManager': {
            'type': 'RoundRobinManagerState',
            'version': '1.0.0',
            'message_thread': [
                {
                    'source': 'user',
                    'models_usage': None,
                    'metadata': {},
                    'content': 'Write a 4-line poem about the ocean.',
                    'type': 'TextMessage'
                },
                {
                    'source': 'writer',
                    'models_usage': {'prompt_tokens': 88, 'completion_tokens': 36},
                    'metadata': {},
                    'content': "Blue depths cradle secrets untold,  \nWhere moonlit waves gently unfold,  \nThe 
ocean's voice, a timeless song,  \nIn its embrace, we find we belong.",
                    'type': 'TextMessage'
                },
                {
                    'source': 'user_proxy',
                    'models_usage': None,
                    'metadata': {},
                    'content': 'make it more funny',
                    'type': 'TextMessage'
                },
                {
                    'source': 'writer',
                    'models_usage': {'prompt_tokens': 139, 'completion_tokens': 39},
                    'metadata': {},
                    'content': "T